In [2]:
import os
import pickle

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
DATA_PATH = '../data/data.csv'
TEST_SIZE = 0.2
STATE = 4

In [4]:
data = pd.read_csv(DATA_PATH)

In [5]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,41,0,1,130,204,0,2,172,0,1.4,0,0,0,0
1,58,0,2,120,340,0,0,172,0,0.0,0,0,0,0
2,63,1,0,145,233,1,2,150,0,2.3,2,0,1,0
3,62,0,3,140,394,0,2,157,0,1.2,1,0,0,0
4,45,1,0,110,264,0,0,132,0,1.2,1,0,2,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        287 non-null    int64  
 1   sex        287 non-null    int64  
 2   cp         287 non-null    int64  
 3   trestbps   287 non-null    int64  
 4   chol       287 non-null    int64  
 5   fbs        287 non-null    int64  
 6   restecg    287 non-null    int64  
 7   thalach    287 non-null    int64  
 8   exang      287 non-null    int64  
 9   oldpeak    287 non-null    float64
 10  slope      287 non-null    int64  
 11  ca         287 non-null    int64  
 12  thal       287 non-null    int64  
 13  condition  287 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 31.5 KB


Пропущенных значений нет.

Посмотрим на статистики

In [7]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
count,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000,287.000000
mean,54.484321,0.679443,2.167247,131.533101,247.477352,0.142857,1.000000,149.728223,0.324042,1.056098,0.599303,0.679443,0.846690,0.459930
std,8.979713,0.467506,0.960718,17.575555,51.941656,0.350538,0.996497,22.933379,0.468833,1.168806,0.617121,0.943340,0.959411,0.499262
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,2.000000,120.000000,212.000000,0.000000,0.000000,135.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,56.000000,1.000000,2.000000,130.000000,243.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,0.000000,0.000000
75%,61.000000,1.000000,3.000000,140.000000,275.000000,0.000000,2.000000,165.500000,1.000000,1.700000,1.000000,1.000000,2.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,3.000000,2.000000,1.000000


Уберем из данных таргет

In [8]:
X = data.drop('condition', axis=1)
y = data['condition']

X.shape, y.shape

((287, 13), (287,))

In [9]:
# откусим кусок на тест
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=10, random_state=STATE, shuffle=True)

In [10]:
X_test.to_csv('test.csv', index=False)
y_test.to_csv('test_target.csv', index=False)

In [11]:
# оставшиеся данные сохраним
X_train['condition'] = y_train
X_train.to_csv('data.csv', index=False)

In [12]:
X_train = X_train.drop('condition', axis=1)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=TEST_SIZE, random_state=STATE, shuffle=True)

In [14]:
categorial_features = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]
numerical_features = ["age", "trestbps", "chol", "thalach", "oldpeak"]
scaler = StandardScaler()
scaler.fit(X_train[numerical_features])

StandardScaler()

In [15]:
X_train[numerical_features] = scaler.transform(X_train[numerical_features])
X_val[numerical_features] = scaler.transform(X_val[numerical_features])

# Случайный лес

In [17]:
forest = RandomForestClassifier(n_estimators=100, random_state=12314, criterion='entropy')
forest.fit(X_train, y_train)
y_forest = forest.predict(X_val)
print(f'Точность для случайного леса: {accuracy_score(y_val, y_forest):.2f}')
print(f'f1_score: {f1_score(y_val, y_forest):.2f}')
print(f'recall_score: {recall_score(y_val, y_forest):.2f}')
print(f'precision_score: {precision_score(y_val, y_forest):.2f}')

Точность для случайного леса: 0.82
f1_score: 0.80
recall_score: 0.80
precision_score: 0.80
